In [1]:
import numpy as np
import pandas as pd

In [2]:
data = pd.read_table("./play.txt", sep=",")
# data['id'] = np.arange(1,15)
data

,outlook,temperature,humidity,windy,play
0,sunny,hot,high,False,no
1,sunny,hot,high,True,no
2,overcast,hot,high,False,yes
3,rainy,mild,high,False,yes
4,rainy,cool,normal,False,yes
5,rainy,cool,normal,True,no
6,overcast,cool,normal,True,yes
7,rainy,mild,normal,False,yes
8,sunny,mild,normal,True,yes
9,overcast,mild,high,True,yes


In [26]:
# #计算信息熵
# def cal_information_entropy(data):
#     data_label = data.iloc[:,-1]
#     label_class =data_label.value_counts() #总共有多少类
#     Ent = 0
#     for k in label_class.keys():
#         p_k = label_class[k]/len(data_label)
#         Ent += -p_k*np.log2(p_k)
#     return Ent

# #计算给定数据属性a的信息增益
# def cal_information_gain(data, a):
#     Ent = cal_information_entropy(data)
#     feature_class = data[a].value_counts() #特征有多少种可能
#     gain = 0
#     for v in feature_class.keys():
#         weight = feature_class[v]/data.shape[0]
#         Ent_v = cal_information_entropy(data.loc[data[a] == v])
#         gain += weight*Ent_v
#     return Ent - gain

In [102]:
# en = []
# ratio = []
# for _, groups in data.groupby("outlook"):
#     display(groups)
#     ratio.append(len(groups) / len(data))
#     values = groups['play'].value_counts(normalize=True).values
#     log_values = np.log2(values)
#     entropy = -1.0 * sum(values * log_values)
#     en.append(entropy)

In [27]:
## 计算给定属性的信息熵
def cal_entropy(data, feature, label):
    en = []
    ratio = []
    for _, groups in data.groupby(feature):
        ratio.append(len(groups) / len(data))
        values = groups[label].value_counts(normalize=True).values
        log_values = np.log2(values)
        entropy = -1.0 * sum(values * log_values)
        en.append(entropy)
    res = sum(np.array(en) * np.array(ratio))
    return res

## 计算标签的信息熵
def cal_entropy_all(data, label):
    values = data[label].value_counts(normalize=True).values
    log_values = np.log2(values) 
    overall_entropy = -1.0 * sum(values * log_values)
    return overall_entropy

## 计算信息增益
def cal_entropy_gain(overall_entropy, feature_entropy):
    return overall_entropy - feature_entropy

In [28]:
overall_entropy = cal_entropy_all(data=data,  label='play')
overall_entropy

0.9402859586706311

In [29]:
outlook_entropy = cal_entropy(data=data, feature='outlook', label='play')
outlook_gain = cal_entropy_gain(overall_entropy, outlook_entropy)
outlook_gain

0.24674981977443933

In [30]:
temperature_entropy = cal_entropy(data=data, feature='temperature', label='play')
temperature_gain = cal_entropy_gain(overall_entropy, temperature_entropy)
temperature_gain

0.02922256565895487

In [31]:
humidity_entropy = cal_entropy(data=data, feature='humidity', label='play')
humidity_gain = cal_entropy_gain(overall_entropy, humidity_entropy)
humidity_gain

0.15183550136234159

In [32]:
windy_entropy = cal_entropy(data=data, feature='windy', label='play')
windy_gain = cal_entropy_gain(overall_entropy, windy_entropy)
windy_gain

0.04812703040826949

In [61]:
## 获取标签最多的那一类
def get_most_label(data, label):
    label_sort = data[label].value_counts(sort=True)
    return label_sort.keys()[0]

get_most_label(data, label = 'play')

'yes'

In [100]:
## 挑选最优特征，即信息增益最大的特征
def get_best_feature(data, label):
    overall_entropy = cal_entropy_all(data=data, label=label)
    features = data.drop(label, axis=1).columns
    gain = {}
    for f in features:
        feature_entropy = cal_entropy(data, feature=f, label=label)
        entropy_gain = cal_entropy_gain(overall_entropy, feature_entropy)
        gain[f] = entropy_gain
    gain_sorted_values = sorted(gain.items(), key=lambda x:x[1], reverse=True)
    return gain_sorted_values[0][0]

In [101]:
get_best_feature(data = data, label = 'play')

'outlook'